# DATA LEMUR QUESTS

This repository contains SQL questions from DataLemur that challenged me or taught me something meaningful along the way.

Some datasets I recreated myself, others I simply printed for reference. The goal here is learning and showcasing practical SQL solutions.

❗ **DataLemur is an amazing platform, so if you're studying SQL, I strongly recommend solving the questions directly on the site. `datatalemur.com`**


Feel free to explore the examples here, and if you're curious about anything, you can reach out. :)

#### Setting the ambient

In [28]:
pip install tabulate

  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("tweets.db")
cursor = conn.cursor()

### Histogram of Tweets - Easy 🟢


Here we calculate each user’s tweet count in 2022, assign them to a bucket based on that count, and then compute how many users fall into each bucket.

#### Creatintg tweets table

In [10]:
cursor.executescript("""
DROP TABLE IF EXISTS tweets;
            
CREATE TABLE tweets (
    tweet_id INTEGER,
    user_id INTEGER,
    msg TEXT,
    tweet_date TEXT
);
""")

conn.commit()

In [11]:
cursor.executescript("""
INSERT INTO tweets VALUES 
(214252, 111, 'Am considering taking Tesla private at $420. Funding secured.', '2021-12-30 00:00:00'),
(739252, 111, 'Despite the constant negative press covfefe', '2022-01-01 00:00:00'),
(846402, 111, 'Following @NickSinghTech on Twitter changed my life!', '2022-02-14 00:00:00'),
(241425, 254, 'If the salary is so competitive why won''t you tell me what it is?', '2022-03-01 00:00:00'),
(231574, 148, 'I no longer have a manager. I can''t be managed', '2022-03-23 00:00:00');
""")

conn.commit()


#### Step by step solution

First I got the number of tweets per user

In [30]:
tweet_query_1 = ("""

SELECT user_id, COUNT(*) as num_tweets 
FROM tweets
WHERE tweet_date BETWEEN '2022-01-01' AND '2022-12-31'
GROUP BY user_id

""")

cursor.execute(tweet_query_1)

users = pd.read_sql(tweet_query_1, conn)
print(users.to_markdown(index=False))

|   user_id |   num_tweets |
|----------:|-------------:|
|       111 |            2 |
|       148 |            1 |
|       254 |            1 |


Then we use it as the core of the main query, in the FROM clause

In [29]:
tweet_query_2 = ("""
SELECT 
    num_tweets as tweet_bucket,
    COUNT(user_id) AS users_num
       
FROM (

SELECT user_id, 
       COUNT(*) as num_tweets
FROM tweets
WHERE tweet_date BETWEEN '2022-01-01' AND '2022-12-31'
GROUP BY user_id

) AS total_tweets

GROUP BY num_tweets
""")

cursor.execute(tweet_query_2)

bucket = pd.read_sql(tweet_query_2, conn)
print(bucket.to_markdown(index=False))


|   tweet_bucket |   users_num |
|---------------:|------------:|
|              1 |           2 |
|              2 |           1 |


Done! ✔️